In [1]:
### dependencies

import numpy as np
from citipy import citipy
from config import weather_api_key
from config import g_key
import gmaps
import requests
import pandas as pd
import csv

In [2]:
### create a new set of 2000 random latitudes and longitudes

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
# creates floating point tuples 'lats' and 'lngs', each filled with random values to be used for coordinates
coordinates = list(zip(lats, lngs))
# creates an object with both tuples paired together, then converts them to a list that we can use
coordinates[:10]
# check our work to see that the list of random coordinate tuples is created properly

[(-22.631295748616594, -177.0643200536824),
 (-45.88309688648343, 101.7846859377562),
 (62.26774412952079, -70.62966820219201),
 (28.37951541733254, 110.35082986943456),
 (32.09347903477685, 102.88236092812292),
 (-2.6613922768882787, -175.91834630612382),
 (55.08043524450892, -152.49295795494504),
 (3.7112121466578287, 160.99132560685354),
 (-68.82536604356272, 8.429817513254108),
 (-61.32452177695325, 97.16101035738711)]

In [3]:
### get the nearest city using the 'citipy' module

cities = []
# create an empty list for holding the city names

## identify the nearest city for each latitude/longitude combination
for coordinate in coordinates:
# for each coordinate in the list 'coordinates',
    city_select = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # find the nearest city to specified coordinates, return the name; assign it the variable 'city'
    if city_select not in cities:
    # if the newly generated city is not already in the list 'cities'
        cities.append(city_select)
        # add the newly generated city to the list 'cities'

len(cities), cities[:10]
# show the length of the 'cities' list to confirm we have enough datapoints, and show first ten entries to confirm precision

(773,
 ['vaini',
  'busselton',
  'iqaluit',
  'jishou',
  'tianpeng',
  'vaitupu',
  'kodiak',
  'kieta',
  'hermanus',
  'rikitea'])

In [4]:
### perform API call with the OpenWeatherMap

url = "https://api.openweathermap.org/data/2.5/weather?lat=35&lon=139&appid=" + weather_api_key
# generates url for weather map API call
print(url)
# prints generated url to check our work

https://api.openweathermap.org/data/2.5/weather?lat=35&lon=139&appid=16d24d2eb2143f2ff1efd161db7f61cb


In [5]:
### retrieve the following from API call: Lat/Long, Max Temp, Percent Humidity, Percent Cloudiness, Wind Speed, Weather Description

city_data = []
# create an empty list to hold our cities' weather data
record_count = 1
set_count = 1
# set counters to 1

print("Beginning Data Retrieval...")
print("---------------------------")
# prints the beginning of our data logging

for i, city in enumerate(cities):
# establishes 'i' and 'city' as variables, assigning them by using enumerate() to return both the index and city name in 'cities'

    ## this bit will separate our list of hundreds of cities, into sections of fifty. Here, it's basically used like:
    ## count out fifty index entries, place a pointer and reset the counter
    if (i % 50 == 0 and i >= 50):
    # if 'index modulus 50' is equivalent to zero and index is greater than or equal to 50, 
        set_count += 1
        record_count = 1
        # increment the 'set_count' by one, and reset the 'record_count' to 1 
        
    city_url = url + "&q=" + city.replace(" ","+")
    # create endpoint URl for the current city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # log the url, record, set numbers, and the city
    record_count += 1
    # increment the record counter
    
    ## this next bit will perform an API request on the current city
    try:
    # use a try/except expression so that the loop does not get hung up on empty coordinate indices
    
        city_weather = requests.get(city_url).json()
        # parse the JSON and retrieve url data for current city
        
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_descript = city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_wind = city_weather["wind"]["speed"]
        city_clouds = city_weather["clouds"]["all"]
        city_country = city_weather["sys"]["country"]
        # assign desired variables from the city data 
        
        city_max_temp = (city_max_temp - 273.15) * 9/5 + 32
        # converts the json temp (kelvin) to fahrenheit
        
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Weather Description": city_descript,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Wind Speed": city_wind,
                          "Cloudiness": city_clouds,
                          "Country": city_country})

    except:
    # if there is no city found (if the coordinate does not have a city near it):
        print("City not found. Skipping...")
        pass
        # print "City not found..." and move to next entry
        
        
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
# once our 'for' loop has iterated through all the entries in the 'cities' list, log that the data retrieval is complete
    

Beginning Data Retrieval...
---------------------------
Processing Record 1 of Set 1 | vaini
Processing Record 2 of Set 1 | busselton
Processing Record 3 of Set 1 | iqaluit
Processing Record 4 of Set 1 | jishou
Processing Record 5 of Set 1 | tianpeng
Processing Record 6 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 7 of Set 1 | kodiak
Processing Record 8 of Set 1 | kieta
Processing Record 9 of Set 1 | hermanus
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | ribeira grande
Processing Record 12 of Set 1 | luganville
Processing Record 13 of Set 1 | provideniya
Processing Record 14 of Set 1 | carnarvon
Processing Record 15 of Set 1 | tessalit
Processing Record 16 of Set 1 | bredasdorp
Processing Record 17 of Set 1 | qaanaaq
Processing Record 18 of Set 1 | yellowknife
Processing Record 19 of Set 1 | airai
Processing Record 20 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 21 of Set 1 | punta arenas
Processing Record 22 of Set 1 

Processing Record 39 of Set 4 | puri
Processing Record 40 of Set 4 | leh
Processing Record 41 of Set 4 | saldanha
Processing Record 42 of Set 4 | isangel
Processing Record 43 of Set 4 | constitucion
Processing Record 44 of Set 4 | veranopolis
Processing Record 45 of Set 4 | constantine
Processing Record 46 of Set 4 | urusha
Processing Record 47 of Set 4 | port macquarie
Processing Record 48 of Set 4 | sibolga
Processing Record 49 of Set 4 | tuggurt
City not found. Skipping...
Processing Record 50 of Set 4 | belushya guba
City not found. Skipping...
Processing Record 1 of Set 5 | puerto ayora
Processing Record 2 of Set 5 | taburi
City not found. Skipping...
Processing Record 3 of Set 5 | erice
Processing Record 4 of Set 5 | kamenskoye
City not found. Skipping...
Processing Record 5 of Set 5 | tres marias
Processing Record 6 of Set 5 | tulagi
Processing Record 7 of Set 5 | varhaug
Processing Record 8 of Set 5 | atuona
Processing Record 9 of Set 5 | knin
Processing Record 10 of Set 5 | sa

City not found. Skipping...
Processing Record 26 of Set 8 | sorong
Processing Record 27 of Set 8 | benghazi
Processing Record 28 of Set 8 | nemuro
Processing Record 29 of Set 8 | iralaya
Processing Record 30 of Set 8 | warrnambool
Processing Record 31 of Set 8 | umzimvubu
City not found. Skipping...
Processing Record 32 of Set 8 | east london
Processing Record 33 of Set 8 | ayer itam
City not found. Skipping...
Processing Record 34 of Set 8 | itamaraca
Processing Record 35 of Set 8 | slave lake
Processing Record 36 of Set 8 | sweetwater
Processing Record 37 of Set 8 | hambantota
Processing Record 38 of Set 8 | vestmanna
Processing Record 39 of Set 8 | igrim
Processing Record 40 of Set 8 | mirik
Processing Record 41 of Set 8 | visby
Processing Record 42 of Set 8 | leshukonskoye
Processing Record 43 of Set 8 | namibe
Processing Record 44 of Set 8 | slobodskoy
Processing Record 45 of Set 8 | talnakh
Processing Record 46 of Set 8 | altay
Processing Record 47 of Set 8 | awbari
Processing Re

Processing Record 17 of Set 12 | nikki
Processing Record 18 of Set 12 | yoichi
Processing Record 19 of Set 12 | petauke
Processing Record 20 of Set 12 | young
Processing Record 21 of Set 12 | pangnirtung
Processing Record 22 of Set 12 | ambilobe
Processing Record 23 of Set 12 | san andres
Processing Record 24 of Set 12 | chauk
Processing Record 25 of Set 12 | bantry
Processing Record 26 of Set 12 | gizo
Processing Record 27 of Set 12 | ocos
Processing Record 28 of Set 12 | lewistown
Processing Record 29 of Set 12 | sobreira
Processing Record 30 of Set 12 | saint-jean-port-joli
Processing Record 31 of Set 12 | moss vale
Processing Record 32 of Set 12 | praia da vitoria
Processing Record 33 of Set 12 | durban
Processing Record 34 of Set 12 | griffin
Processing Record 35 of Set 12 | helong
Processing Record 36 of Set 12 | knoxville
Processing Record 37 of Set 12 | kaliua
Processing Record 38 of Set 12 | ust-kamchatsk
City not found. Skipping...
Processing Record 39 of Set 12 | progreso
Pr

Processing Record 3 of Set 16 | bud
Processing Record 4 of Set 16 | farcasesti
Processing Record 5 of Set 16 | nortelandia
Processing Record 6 of Set 16 | rondonopolis
Processing Record 7 of Set 16 | banjar
Processing Record 8 of Set 16 | kargasok
Processing Record 9 of Set 16 | kyabe
Processing Record 10 of Set 16 | yuancheng
Processing Record 11 of Set 16 | san vicente
Processing Record 12 of Set 16 | agua verde
Processing Record 13 of Set 16 | khorixas
Processing Record 14 of Set 16 | svetlyy
Processing Record 15 of Set 16 | russell
Processing Record 16 of Set 16 | savonlinna
Processing Record 17 of Set 16 | acapulco
Processing Record 18 of Set 16 | doka
Processing Record 19 of Set 16 | mangaluru
Processing Record 20 of Set 16 | turtkul
City not found. Skipping...
Processing Record 21 of Set 16 | mampong
Processing Record 22 of Set 16 | kawalu
Processing Record 23 of Set 16 | asosa
-----------------------------
Data Retrieval Complete      
-----------------------------


In [6]:
print(len(city_data))
# check that the list was created successfully

712


In [7]:
### add the data to a new dataframe

city_data_df = pd.DataFrame(city_data)
# put the 'city_data' list data into a new dataframe

In [8]:
### show the first ten rows of the dataframe

city_data_df.head(10)
# checks our work

,City,Lat,Lng,Weather Description,Max Temp,Humidity,Wind Speed,Cloudiness,Country
0,Vaini,-21.2000,-175.2000,few clouds,80.762,74,3.60,20,TO
1,Busselton,-33.6500,115.3333,overcast clouds,63.950,59,6.86,98,AU
2,Iqaluit,63.7506,-68.5145,broken clouds,50.738,57,2.44,64,CA
3,Jishou,28.3167,109.7167,overcast clouds,80.420,70,1.06,99,CN
4,Tianpeng,30.9866,103.9393,overcast clouds,79.718,57,2.00,100,CN
5,Kodiak,57.7900,-152.4072,moderate rain,51.314,87,6.17,100,US
6,Kieta,-6.2167,155.6333,few clouds,84.902,66,1.76,11,PG
7,Hermanus,-34.4187,19.2345,light rain,46.796,72,4.53,37,ZA
8,Rikitea,-23.1203,-134.9692,overcast clouds,73.166,70,3.95,96,PF
9,Ribeira Grande,38.5167,-28.7000,overcast clouds,64.814,94,4.84,92,PT


In [9]:
### export the dataframe to a .csv file 

city_data_df.to_csv(r"WeatherPy_Database.csv", index=False)